In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 12:47:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd  
import random

In [3]:
def reservoir_sampling(rdd, k=50000):
    reservoir = []
    for i, item in enumerate(rdd.toLocalIterator()):
        if i < k:
            reservoir.append(item)
        else:
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir

In [4]:
# Read the CSV files
df = spark.read.option("header", "true").csv("../shared/midterm/drive_stats_2019_Q1/*.csv")

In [5]:
df.printSchema()
df.head()

root
 |-- date: string (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: string (nullable = true)
 |-- failure: string (nullable = true)
 |-- smart_1_normalized: string (nullable = true)
 |-- smart_1_raw: string (nullable = true)
 |-- smart_2_normalized: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_normalized: string (nullable = true)
 |-- smart_3_raw: string (nullable = true)
 |-- smart_4_normalized: string (nullable = true)
 |-- smart_4_raw: string (nullable = true)
 |-- smart_5_normalized: string (nullable = true)
 |-- smart_5_raw: string (nullable = true)
 |-- smart_7_normalized: string (nullable = true)
 |-- smart_7_raw: string (nullable = true)
 |-- smart_8_normalized: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_normalized: string (nullable = true)
 |-- smart_9_raw: string (nullable = true)
 |-- smart_10_normalized: string (nullable = tru

Row(date='2019-03-05', serial_number='Z305B2QN', model='ST4000DM000', capacity_bytes='4000787030016', failure='0', smart_1_normalized='117', smart_1_raw='166138696', smart_2_normalized=None, smart_2_raw=None, smart_3_normalized='91', smart_3_raw='0', smart_4_normalized='100', smart_4_raw='12', smart_5_normalized='100', smart_5_raw='0', smart_7_normalized='85', smart_7_raw='374504040', smart_8_normalized=None, smart_8_raw=None, smart_9_normalized='68', smart_9_raw='28210', smart_10_normalized='100', smart_10_raw='0', smart_11_normalized=None, smart_11_raw=None, smart_12_normalized='100', smart_12_raw='12', smart_13_normalized=None, smart_13_raw=None, smart_15_normalized=None, smart_15_raw=None, smart_16_normalized=None, smart_16_raw=None, smart_17_normalized=None, smart_17_raw=None, smart_22_normalized=None, smart_22_raw=None, smart_23_normalized=None, smart_23_raw=None, smart_24_normalized=None, smart_24_raw=None, smart_168_normalized=None, smart_168_raw=None, smart_170_normalized=None

In [6]:
df.count()

9577046

In [7]:
df.show(5)

+----------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-

In [8]:
# Convert DataFrame to RDD
rdd = df.rdd

# Skip header row
header = rdd.first()
rdd = rdd.filter(lambda row: row != header)

In [9]:
# Perform Reservoir Sampling
sample = reservoir_sampling(rdd, k=50000)

In [10]:
len(sample)

50000

In [11]:
df = pd.DataFrame(sample) 
df.to_csv('output.csv')